## Homework 2: Search hyperparameters for neural networks
In this homework, you will practice using Keras to implement neural networks and search their hyperparameters for handwritten digits classification.

### Dataset
MNIST handwritten digits. 60k 28*28 grayscale training images of the 10 digits, along with a test set of 10k images

![Fig. 1](handwrittendigits.png)
 *Fig.1. Handwritten digits examples.* 

### Instructions

    1. You need to install Keras on your computer to build neural networks.
    2. The framework, e.g., functions' names, input, and output, has been defined. You are going to complete the create_NN, nn_params_search, retrain_best_nn, myEvaluation functions.
    3. Add your code in the following blocks, and do not change other places.

```python

    ## add your code here
    
    ##
```

### Student information
    1. Your name: Carlos Santos
    2. Department: Comp. E.
    3. Undergraduate

### Task points and TA grading: ??/100
    1. ?/10
    2. ?/30
    3. ?/30
    4. ?/10
    5. ?/20
    6: performance_acc: ?/10

In [1]:
import keras
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import numpy as np

### 1. Load the MNIST dataset in Keras. 10 points

In [2]:
import matplotlib.pyplot as plt
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

#Task 1. load the dataset
(X_train_2D, y_train, X_test_2D, y_test) = load_data()

# 1.1 reshape the training and test sets (N*28*28) to N * 784. 10 points
X_train = np.reshape(X_train_2D, (X_train_2D.shape[0], (X_train_2D.shape[1]*X_train_2D.shape[2])))
X_test = np.reshape(X_test_2D, (X_test_2D.shape[0], (X_test_2D.shape[1]*X_test_2D.shape[2])))

print('X_train_2D: {}, X_train: {}'.format(X_train_2D.shape, X_train.shape))
print('X_test_2D: {}, X_test: {}'.format(X_test_2D.shape, X_test.shape))

# 1.2 transform y_train to one-hot vectors using keras.utils.to_categorical to form the output of the NN. 10 points
y_train_onehot = keras.utils.to_categorical(y_train) 
print('y_train_onehot: {}'.format(y_train_onehot.shape))
print('one-hot vector examples:\n', y_train_onehot[:5])

X_train_2D: (60000, 28, 28), X_train: (60000, 784)
X_test_2D: (10000, 28, 28), X_test: (10000, 784)
y_train_onehot: (60000, 10)
one-hot vector examples:
 [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2. Create neural networks using Keras. 30 points

In [3]:
def create_NN(hidden_layers = [1000], act = 'relu', opt = 'rmsprop'): 
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    hidden_layers: a list that defines the numbers of hidden nodes for all hidden layers, e.g., [1000] indicates
    the nn has only one hidden layer with 1000 nodes, while [1000, 500] defines two hidden layers and the first
    layer has 1000 nodes and the second has 500 nodes.
    act: activation function for all hidden layers
    opt: optimizer
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    in_dim = 784
    out_dim = 10    
    myNN = keras.models.Sequential()
    
    
    num_hidden_layers = 0
    for layer in hidden_layers:
        num_hidden_layers = num_hidden_layers + 1 
        print("Hidden Layer:", num_hidden_layers, "Size of:", layer)
        #2.1 build all hidden layers
        myNN.add(keras.layers.Dense(
        units=layer,
        input_dim=in_dim,
        activation=act))

    #2.2 build the output layer and use the softmax activation  
    myNN.add(keras.layers.Dense(
    units=out_dim,
    activation='softmax'))

    #2.3 choose the optimizer, compile the network and return it. Use 'accuracy' as the metrics
    myNN.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return myNN
    
h_nodes = [1000, 500] # two hidden layers with 1000 and 500 nodes, respectively.
myNN = create_NN(hidden_layers= h_nodes, act = 'relu', opt = 'adam')
myNN.summary()

Hidden Layer: 1 Size of: 1000
Hidden Layer: 2 Size of: 500
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              785000    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5010      
Total params: 1,290,510
Trainable params: 1,290,510
Non-trainable params: 0
_________________________________________________________________


### 3. Search the best NN paprameters, and report the performance. 30 points
    - The KerasClassifier will be used to warp NN models to use the GridSearchCV
    - Complete the nn_params_search function to search the three parameters: batch_size, activation, and optimizer.
    - Each fit (10 epochs) may take 1 to 2 minutes.

In [4]:
from keras.wrappers.scikit_learn import KerasClassifier

def nn_params_search(nn, X, y, param_grid): # 30 points
    '''Search best paramaters
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input
    param_grid: a dict that defines the parameters
    
    Returns
    -------
    best_params_
        
    '''
    ## add your code here. set cv = 3, scoring = 'accuracy', and verbose = 2
    grid = GridSearchCV(estimator=nn, cv = 3, scoring = 'accuracy', verbose = 2, param_grid=param_grid)
    results = grid.fit(X, y)
    return results.best_params_
    
    ##

# wrap keras model to use in sklearn
nn = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 10) # using the keras wapper
param_grid = {'batch_size': [64, 128], 
              'act':['relu', 'sigmoid'], 'opt': ['sgd', 'adam']}

best_params = nn_params_search(nn, X_train, y_train, param_grid = param_grid)
print('\nBest parameters: ', best_params)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 3s 5ms/step - loss: 797.0479 - accuracy: 0.5154
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.8903 - accuracy: 0.7760
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.7006 - accuracy: 0.8200
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6673 - accuracy: 0.8278
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.6286 - accuracy: 0.8119
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5718 - accuracy: 0.8376
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5675 - accuracy: 0.8733
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4841 - accuracy: 0.8820
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5535 - accuracy: 0.8202
Epoch 10

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ...................act=relu, batch_size=64, opt=sgd; total time=  31.4s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 436.9392 - accuracy: 0.6089
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5345 - accuracy: 0.8689
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.4478 - accuracy: 0.9010
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4553 - accuracy: 0.9029
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3593 - accuracy: 0.9192
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3510 - accuracy: 0.9208
Epoch 7/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3072 - accuracy: 0.9260
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2463 - accuracy: 0.9451
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2537 - accu

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ...................act=relu, batch_size=64, opt=sgd; total time=  34.7s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 305.9316 - accuracy: 0.6520
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5293 - accuracy: 0.8801
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3940 - accuracy: 0.9094
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3575 - accuracy: 0.9209
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3000 - accuracy: 0.9327
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2756 - accuracy: 0.9366
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2841 - accuracy: 0.9396
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2534 - accuracy: 0.9475
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2245 - accu

...

[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  31.6s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
313/313 [==============================] - 3s 8ms/step - loss: 0.5665 - accuracy: 0.8386
Epoch 2/10
313/313 [==============================] - 2s 8ms/step - loss: 0.2535 - accuracy: 0.9321
Epoch 3/10
313/313 [==============================] - 2s 8ms/step - loss: 0.2166 - accuracy: 0.9389
Epoch 4/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2024 - accuracy: 0.9430
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1988 - accuracy: 0.9442
Epoch 6/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1893 - accuracy: 0.9475
Epoch 7/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1740 - accuracy: 0.9508
Epoch 8/10
313/313 [==============================] - 2s 8ms/step - loss: 0.1712 - accuracy: 0.9495
Epoch 9/10
313/313 [==============================] - 2s 8ms/step - loss: 0.1649 - accura

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  26.6s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
313/313 [==============================] - 3s 8ms/step - loss: 0.6107 - accuracy: 0.8198
Epoch 2/10
313/313 [==============================] - 2s 8ms/step - loss: 0.2550 - accuracy: 0.9313
Epoch 3/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2255 - accuracy: 0.9355
Epoch 4/10
313/313 [==============================] - 3s 9ms/step - loss: 0.2083 - accuracy: 0.9422
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1904 - accuracy: 0.9451
Epoch 6/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1837 - accuracy: 0.9478
Epoch 7/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1888 - accuracy: 0.9448
Epoch 8/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1732 - accuracy: 0.9497: 0s - l
Epoch 9/10
313/313 [==============================] - 3s 9ms/step - loss: 0.1687 

C:\Users\carlo\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[CV] END ..............act=sigmoid, batch_size=128, opt=adam; total time=  28.2s
Hidden Layer: 1 Size of: 1000
Epoch 1/10
938/938 [==============================] - 7s 7ms/step - loss: 9.6120 - accuracy: 0.8721
Epoch 2/10
938/938 [==============================] - 6s 6ms/step - loss: 0.3533 - accuracy: 0.9562
Epoch 3/10
938/938 [==============================] - 6s 6ms/step - loss: 0.3008 - accuracy: 0.9584
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 0.2393 - accuracy: 0.9654
Epoch 5/10
938/938 [==============================] - 7s 8ms/step - loss: 0.2417 - accuracy: 0.9629
Epoch 6/10
938/938 [==============================] - 7s 7ms/step - loss: 0.2470 - accuracy: 0.9637
Epoch 7/10
938/938 [==============================] - 6s 7ms/step - loss: 0.2702 - accuracy: 0.9659
Epoch 8/10
938/938 [==============================] - 6s 6ms/step - loss: 0.1803 - accuracy: 0.9704
Epoch 9/10
938/938 [==============================] - 6s 7ms/step - loss: 0.1980 - accura

### 4. Retrain a neural network using the best parameters. 10 points
    - Compelte the retrain_test_nn function to create (create_NN) and train (fit) a new nn using parameters in best_params
    - The default epoches are 20 and each epoch may take around 10 to 20 seconds; and you can increase this value to get better results

In [7]:
def retrain_best_nn(best_params, X_train, y_train, epochs = 10): # 10 points
    '''Retrain a nn using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input (one-hot vectors)
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    ## add your code here  
    h_nodes = [1000, 500] # two hidden layers with 1000 and 500 nodes, respectively.
    myNN = create_NN(hidden_layers= h_nodes, act = best_params["act"], opt = best_params["opt"])
    myNN.summary()
    results = myNN.fit(X_train, y_train, epochs=epochs, batch_size=best_params["batch_size"])
    return myNN

bestNN = retrain_best_nn(best_params, X_train, y_train_onehot, epochs = 20)

Hidden Layer: 1 Size of: 1000
Hidden Layer: 2 Size of: 500
Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dense_60 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_61 (Dense)             (None, 10)                5010      
Total params: 1,290,510
Trainable params: 1,290,510
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
938/938 [==============================] - 10s 10ms/step - loss: 7.9892 - accuracy: 0.8638
Epoch 2/20
938/938 [==============================] - 10s 11ms/step - loss: 0.1752 - accuracy: 0.9566
Epoch 3/20
938/938 [==============================] - 10s 10ms/step - loss: 0.1271 - accuracy: 0.9657
Epoch 4/

### 5. Network evaluation. 20 points

    - Complete the myEvaluation function to report the performance of your best nn using the test set
        - compute the overall accuracy
        - compute the precision for each class

In [ ]:
from sklearn.metrics import classification_report

def myEvaluation(y, y_pred):
    ''' calculate the overall accuracy and precision
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        acc: accuracy
        precision: precision array
    '''
    ## add your code here
    # calculate the overall acc

    
    # calculate the precision for each class

    
    # return acc and precision array
    
    
    ##

#
y_test_pred = bestNN.predict_classes(X_test)
acc, precision = myEvaluation(y_test, y_test_pred)
print('my accuracy:    {:.2f}'.format(acc))
print('my precision:', precision)

# results calculated using the classification_report
print(classification_report(y_test, y_test_pred))

### 6. Any findings or conclusion, e.g., useful strategies to improve nn performance. Extra 10 points
1.

2.

3.

4....
